# Black-Scholes-Merton Model For Estimation of Probability of Default

The famous BSM model can be used to estimate the probability of default of an entity. We need the following parameters for the model,

- Volatility of the asset value (σ)
- Growth rate (μ)
- Account Balance or Net Worth (V)
- Time horizon (T)

## Estimating the Growth Rate and Volatility

To estimate the growth rate and volatility, we can use the previous history of transactions (withdrawals and deposits) to calculate them.

1. Calculate Single_Point_Net_Cash_Flow (SPNCF) Column = deposit_amt - withdrawal_amt 
2. Calculate Cumulative Asset Value (CAV) Column
3. Calculate Daily Growth Rate (DGR) Column = `log(present_asset_value/previous_asset_value)`

After calculating these additional columns, we got,
- Volatility as the standard deviation of the DGR
- Growth Rate as the mean of the DGR

In [ ]:
import pandas as pd
import numpy as np

# Correct dataset without ellipsis
data = pd.read_csv('../datasets/bank.csv')

data.head()

# Preprocess numeric columns
data["WITHDRAWAL_AMT"] = data["WITHDRAWAL_AMT"].fillna(0)
data["DEPOSIT_AMT"] = data["DEPOSIT_AMT"].fillna(0)

# Calculate net cash flow
data["NET CASH FLOW"] = data["DEPOSIT_AMT"] - data["WITHDRAWAL_AMT"]

# Calculate cumulative asset value (starting with an initial value, e.g., 10,000,000)
data["ASSET VALUE"] = data["NET CASH FLOW"].cumsum()

# Calculate daily growth rates
data["GROWTH RATE"] = np.log(data["ASSET VALUE"] / data["ASSET VALUE"].shift(1))

# Drop NaN values caused by shift
data = data.dropna(subset=["GROWTH RATE"])

# Calculate volatility and growth rate
volatility = data["GROWTH RATE"].std()
growth_rate = data["GROWTH RATE"].mean()

print(data)
print(f"Volatility (σ): {volatility}")
print(f"Growth Rate (μ): {growth_rate}")

             DATE  WITHDRAWAL_AMT  DEPOSIT_AMT  NET CASH FLOW   ASSET VALUE  \
1       05-Jul-17            0.00    1000000.0     1000000.00  2.000000e+06   
2       18-Jul-17            0.00     500000.0      500000.00  2.500000e+06   
3       01-Aug-17            0.00    3000000.0     3000000.00  5.500000e+06   
4       16-Aug-17            0.00     500000.0      500000.00  6.000000e+06   
5       16-Aug-17            0.00     500000.0      500000.00  6.500000e+06   
...           ...             ...          ...            ...           ...   
116196  05-Mar-19       117934.30          0.0     -117934.30 -1.901902e+09   
116197  05-Mar-19            0.00     300000.0      300000.00 -1.901602e+09   
116198  05-Mar-19            0.00     300000.0      300000.00 -1.901302e+09   
116199  05-Mar-19       109868.65          0.0     -109868.65 -1.901412e+09   
116200  05-Mar-19         5000.00          0.0       -5000.00 -1.901417e+09   

        GROWTH RATE  
1          0.693147  
2      

/home/orientsvictor/grass/.env/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Calculating Probability of Default (PD) using BSM Model

In [25]:
import numpy as np
from scipy.stats import norm

# Parameters
A_0 = 100000  # Initial asset value (example)
D = 80000     # Debt value (example)
T = 1             # Time horizon in years
mu = -0.8  # Mean growth rate (calculated earlier)
sigma = 0.5  # Volatility (calculated earlier)

# Calculate d2
d2 = (np.log(A_0 / D) + (mu - (sigma**2) / 2) * T) / (sigma * np.sqrt(T))

# Calculate Probability of Default (PD)
PD = norm.cdf(-d2)

print(f"d2: {d2}")
print(f"Probability of Default (PD): {PD}")

d2: -1.4037128973715807
Probability of Default (PD): 0.9197978198489728
